In [89]:
import re

f = open('input')
f = f.readlines()
f = [line.strip() for line in f]

root = {}
pwd = root
pwd_path = []

def handle_line(line, directory):
  if line[0] == '$':
    cmd = line[2:4]
    if cmd == "cd":

      target = line[5:]

      if target == '/':
        return ("GO_TO_ROOT", None)
      if target == '..': 
        return ("GO_TO_PARENT", None)

      if not target in directory:
        return ("MAKE_AND_NAVIGATE", target)

      return ("NAVIGATE", target)
    
    if cmd == "ls":
      return ("LIST_FILES", None)
  
  if line[0:3] == 'dir':
    dirname = line[4:]
    # print(dirname)
    return ("MAKE_DIR", dirname)
  
  filesize = re.findall(r'[0-9]+', line)[0]
  filename = re.findall(r'[a-z.]+', line)[0]
  return ("MAKE_FILE", [filename, filesize])

def get_dir(path):
  cdir = root
  for d in path:
    cdir = cdir[d]
  return cdir

for line in f:
  action, param = handle_line(line, pwd)
  if action == "NAVIGATE":
    pwd = pwd[param]
    pwd_path.append(param)

  if action == "MAKE_AND_NAVIGATE":
    pwd[param] = {}
    pwd_path.append(param)
    pwd = pwd[param]

  if action == 'GO_TO_PARENT':
    pwd = get_dir(pwd_path[:-1])
    pwd_path = pwd_path[:-1]

  if action == 'GO_TO_ROOT':
    pwd = root
    pwd_path = []
  
  if action == 'MAKE_DIR':
    pwd[param] = {}

  if action == 'MAKE_FILE':
    filename = param[0]
    filesize = int(param[1])

    # if 'size' in pwd:
    #   pwd['size'] = pwd['size'] + filesize
    # else: 
    #   pwd['size'] = filesize
    
    pwd[filename] = filesize

print(root)


{'cvt': {'bbgsthsd': {'djfww.fcb': 236957, 'hcqbmwc.gts': 112286, 'qggjrzts': 106102}, 'bcqrmp.czf': 146042, 'chhdjtlw': {'cpcfcc': 226927, 'djfww': 309815, 'hcqbmwc.gts': 117933, 'mbdrgfzs': {'hvfvt.dvq': 164331}, 'pbmcnpzf': {'bjgg.cqd': 102120}, 'pwgswq.fld': 131558, 'qlqqmndd.zcb': 298691}, 'cpcfcc': {'ddc': 15756, 'dqc': {'cpcfcc': {'hvfvt': {'pwgswq.fld': 289372, 'qlqqmndd.zcb': 307397}}, 'mzp': 92063, 'qmhnvmh': {'jczjwdl.smd': 274778, 'ngrrfm': 194322}, 'snqqcjlw': {'nprvrgd.tbb': 182851}, 'zdjwr.blc': 8423}, 'glm': {'hpq.hgm': 188702}, 'jbszm': {'pwgswq.fld': 187645}, 'pwgswq.fld': 200345, 'qlqqmndd.zcb': 145508, 'vcptbw': {'mgqd': {'rhldvntj.jzm': 157695, 'rjnngctw': 268696}}, 'zrtm': {'bcqrmp.czf': 218820, 'czpsl.dnf': 172319, 'hcqbmwc.gts': 180114, 'hvfvt.nnw': 165216, 'zft': {'dtqjg.czv': 161245, 'hpq.hgm': 249888, 'zft': 192037}}}, 'dch': {'djfww': {'dct': 37427, 'hcqbmwc.gts': 204401, 'jmq': 283140, 'lbrhbc': {'djfww': {'hcqbmwc.gts': 24794}, 'djfww.pps': 153648, 'mvtjzp

In [74]:
def getdirs(directory):
  directories = []
  for file in directory:
    if type(directory[file]) == dict:
      directories.append(file)
  return directories
print(getdirs(root))

['cvt', 'hvfvt', 'phwgv', 'svw', 'zft']


In [131]:

def calcDirSize(directory):
  dirsize = 0
  accumulated = 0
  for file in directory:
    if type(directory[file]) == dict:
      subdirsize, accumulated_small_dir_size = calcDirSize(directory[file])
      accumulated += accumulated_small_dir_size
      dirsize += subdirsize
    if type(directory[file]) == int:
      dirsize += directory[file]

  if dirsize <= 100000:
    accumulated += dirsize

  return (dirsize, accumulated)

root_size, small_dir_size = calcDirSize(root)

print(f'root size was {root_size}. Dirs below 100000 was {small_dir_size}')

root size was 46728267. Dirs below 100000 was 1390824


In [153]:
total_space = 70000000
needed_space = 30000000
remaining_space = total_space - root_size

clean_up_quota = needed_space - remaining_space

print({"root_size".ljust(20): str(root_size).rjust(9)})
print({"remaining_space".ljust(20): str(remaining_space).rjust(9)})
print({"needed_space".ljust(20): str(needed_space).rjust(9)})
print({"clean_up_quota".ljust(20): str(clean_up_quota).rjust(9)})


def findSmallestCleanupDir(directory):
  dirsize = 0
  closest = 999999999
  for file in directory:
    if type(directory[file]) == dict:
      subdirsize, closest_subdir = findSmallestCleanupDir(directory[file])
      if closest > closest_subdir and closest_subdir > clean_up_quota:
        closest = closest_subdir
      dirsize += subdirsize
    if type(directory[file]) == int:
      dirsize += directory[file]

  if closest > dirsize and dirsize > clean_up_quota:
    closest = dirsize

  return (dirsize, closest)

smallest = findSmallestCleanupDir(root)

print(f'\n✨ Smallest was {smallest[1]} ✨')

{'root_size           ': ' 46728267'}
{'remaining_space     ': ' 23271733'}
{'needed_space        ': ' 30000000'}
{'clean_up_quota      ': '  6728267'}

✨ Smallest was 7490863 ✨
